<a href="https://colab.research.google.com/github/mdcdxcvi/imersao-dados-desafio-final/blob/main/Notebooks/Vin%C3%ADcius_O_Romano_Silva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Imersão Dados
#### Alura


### Carregando conjunto de dados 

In [12]:
import pandas as pd


# Carrega as duas bases de dados
dados_experimento = pd.read_csv('https://github.com/mdcdxcvi/imersao-dados-desafio-final/blob/main/Dados/dados_experimentos.zip?raw=true', compression='zip')
dados_resultados = pd.read_csv('https://github.com/mdcdxcvi/imersao-dados-desafio-final/blob/main/Dados/dados_resultados.csv?raw=true')

### Tratando conjuntos e os unindo em um mesmo conjunto.

In [13]:
# Tratamento das colunas do DataFrame dados_experimento
dados_experimento.rename(columns={'droga':'composto'}, inplace=True)


# Tratamento do DataFrame dados_resultados
def seleciona_reacoes(linha_dados: pd.core.series.Series) -> list:
    """Limpa as reações não registradas em cada coluna.
    
    argumentos:
    linha_dados -- linha do df em formato série
    """
    return linha_dados.to_frame().dropna(axis=0).index.values.tolist()

reacoes = pd.DataFrame(
    dados_resultados.iloc[:,1:][dados_resultados.iloc[:,1:]==1].apply(
        lambda x: seleciona_reacoes(x),
        axis='columns',
    ),
    columns=['reacao_observada'],
)
reacoes['id'] = dados_resultados.iloc[:,0]


# Une as duas bases de dados
dados = pd.merge(
    dados_experimento,
    reacoes,
    on='id'
)

dados.head(5)

,id,tratamento,tempo,dose,composto,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g

### Plot do gráfico para visualização das expressões

In [14]:
import plotly.express as px
from ipywidgets import interactive


# Gráfico interativo para visualização das alterações nos indicadores dado o index da base de dados.
def plot_vizualizacao(indice: int = 256) -> None:
    """Gráfico das alterações em relação ao index da entrada nos dados.
    
    argumentos:
    indice -- index da entrada nos dados (padrão: 256)
    """
    exp_genicas_e_alteracoes = dados.iloc[indice].loc['g-0':'c-99'].to_frame().T
    fig = px.bar(
        dados.iloc[indice].loc['g-0':'c-99'],
        x=exp_genicas_e_alteracoes.columns,
        y=exp_genicas_e_alteracoes.values.tolist()[0],
        title=f'{dados.id[indice]}; tratamento: {dados["tratamento"][indice]}; tempo: {dados.tempo[indice]}; dose: {dados.dose[indice]}; composto: {dados.composto[indice]}.',
        color=exp_genicas_e_alteracoes.values.tolist()[0],
        color_continuous_scale=px.colors.sequential.Bluered_r,
        labels={
            'x': 'Indicador',
            'y': 'Alteração',
            'color': 'cor',
        },
        width=1200,
        range_y=[-10,10],
    )

    reacao = dados.reacao_observada[indice]
    if reacao == []:
        display(pd.DataFrame(data=['Nenhuma'],columns=['Reação']).T)
    elif len(reacao) == 1:
        display(pd.DataFrame(data=reacao,columns=['Reação']).T)
    else:
        display(pd.DataFrame(data=reacao,columns=['Reações']).T)

    fig.show()
 

interactive_plot = interactive(plot_vizualizacao, indice=(0, dados.shape[0] - 1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=256, description='indice', max=23813), Output()), _dom_classes=('widget-…

## Aprendizado de Máquina

### Tratamento dos dados para aplicação do modelo de aprendizagem Árvore de Decisão

In [15]:
import copy


# Tratando o DataFrame dados_resultados para etapa de Aprendizado de Máquina
dados_resultados_tratados_completos = pd.DataFrame(data=dados_resultados['id'])

for alvo_mec_acao in dados_resultados.iloc[:,1:].columns:
    if alvo_mec_acao.split('_')[-1] in ['inhibitor', 'agonist', 'antagonist', 'activator']:
        coluna_alvo = '_'.join(alvo_mec_acao.split('_')[:-1])
        alvo_mec_acao_alvo = alvo_mec_acao.split('_')[-1]
        if coluna_alvo not in dados_resultados_tratados_completos.columns:
            dados_resultados_tratados_completos[coluna_alvo] = 0

        dados_resultados_tratados_completos.loc[dados_resultados[alvo_mec_acao] == 1, coluna_alvo] = alvo_mec_acao_alvo

    elif alvo_mec_acao not in dados_resultados_tratados_completos.columns:
        dados_resultados_tratados_completos[alvo_mec_acao] = 0
        dados_resultados_tratados_completos.loc[dados_resultados[alvo_mec_acao] == 1, alvo_mec_acao] = 'apresenta'

dados_resultados_tratados_completos.replace(to_replace={0:'NA'}, inplace=True)
dados_resultados_tratados = copy.copy(dados_resultados_tratados_completos)

### Criando modelo e treinando
Pode levar por volta de 5 minutos. Espero que valha a pena 😉

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


# Modelo de Árvore de decisão
def criar_listas_vazias(quantas: int) -> list:
    """Cria uma quantidade de lista de listas vazias.
      
    argumentos:
    quantas -- quantas listas vazias devem ser criadas
    """
    return [ [] for _ in range(len(quantas)) ]


profund = [2, 5]
alvos_mec_acao_observados = dados_resultados_tratados.loc[:,'5-alpha_reductase':'acetylcholine_receptor'].columns
teste_g = pd.DataFrame(index=range(profund[0],profund[1]))
teste_c = pd.DataFrame(index=range(profund[0],profund[1]))
teste_gc = pd.DataFrame(index=range(profund[0],profund[1]))
treino_g = pd.DataFrame(index=range(profund[0],profund[1]))
treino_c = pd.DataFrame(index=range(profund[0],profund[1]))
treino_gc = pd.DataFrame(index=range(profund[0],profund[1]))

to_drop = []
for alvo_mec_acao_obs in alvos_mec_acao_observados:
    if min(dados_resultados_tratados[alvo_mec_acao_obs].value_counts()) < 2:
        to_drop.append(alvo_mec_acao_obs)

alvos_mec_acao_observados = alvos_mec_acao_observados.drop(to_drop)
dados_resultados_tratados.drop(to_drop, inplace=True)


# Desenvolvimento e treino do modelo
modelos_g = criar_listas_vazias(alvos_mec_acao_observados)
modelos_c = criar_listas_vazias(alvos_mec_acao_observados)
modelos_gc = criar_listas_vazias(alvos_mec_acao_observados)
X_treinos_g = criar_listas_vazias(alvos_mec_acao_observados)
X_treinos_c = criar_listas_vazias(alvos_mec_acao_observados)
X_treinos_gc = criar_listas_vazias(alvos_mec_acao_observados)
X_testes_g = criar_listas_vazias(alvos_mec_acao_observados)
X_testes_c = criar_listas_vazias(alvos_mec_acao_observados)
X_testes_gc = criar_listas_vazias(alvos_mec_acao_observados)
y_treinos_g = criar_listas_vazias(alvos_mec_acao_observados)
y_treinos_c = criar_listas_vazias(alvos_mec_acao_observados)
y_treinos_gc = criar_listas_vazias(alvos_mec_acao_observados)
y_testes_g = criar_listas_vazias(alvos_mec_acao_observados)
y_testes_c = criar_listas_vazias(alvos_mec_acao_observados)
y_testes_gc = criar_listas_vazias(alvos_mec_acao_observados)


# Treinamento dos modelos
# Pode levar uns três minutos
for alvo_mec_acao_obs in alvos_mec_acao_observados:
    X_g = dados_experimento.loc[:,'g-0':'g-771'].select_dtypes('float64')
    X_c = dados_experimento.loc[:,'c-0':'c-99'].select_dtypes('float64')
    X_gc = dados_experimento.loc[:,'g-0':'c-99'].select_dtypes('float64')
    y = dados_resultados_tratados[alvo_mec_acao_obs].to_frame()
    X_treino_g, X_teste_g, y_treino_g, y_teste_g = train_test_split(
        X_g,
        y,
        test_size=0.2,
        stratify=y,
        random_state=0,
    )
    X_treino_c, X_teste_c, y_treino_c, y_teste_c = train_test_split(
        X_c,
        y,
        test_size=0.2,
        stratify=y,
        random_state=0,
    )
    X_treino_gc, X_teste_gc, y_treino_gc, y_teste_gc = train_test_split(
        X_gc,
        y,
        test_size=0.2,
        stratify=y,
        random_state=0,
    )
    X_treinos_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_treino_g)
    X_treinos_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_treino_c)
    X_treinos_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_treino_gc)
    X_testes_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_teste_g)
    X_testes_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_teste_c)
    X_testes_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_teste_gc)
    y_treinos_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_treino_g)
    y_treinos_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_treino_c)
    y_treinos_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_treino_gc)
    y_testes_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_teste_g)
    y_testes_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_teste_c)
    y_testes_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_teste_gc)
    teste_g[alvo_mec_acao_obs] = 0
    teste_c[alvo_mec_acao_obs] = 0
    teste_gc[alvo_mec_acao_obs] = 0
    treino_g[alvo_mec_acao_obs] = 0
    treino_c[alvo_mec_acao_obs] = 0
    treino_gc[alvo_mec_acao_obs] = 0

    for i in range(profund[0],profund[1]):
        modelo_arvore = DecisionTreeClassifier(max_depth=i)
        modelo_arvore.fit(X_treino_g, y_treino_g)
        modelos_g[i-profund[0]].append(modelo_arvore)
        treino_g.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_treino_g, y_treino_g)
        teste_g.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_teste_g, y_teste_g)
        
        modelo_arvore = DecisionTreeClassifier(max_depth=i)
        modelo_arvore.fit(X_treino_c, y_treino_c)
        modelos_c[i-profund[0]].append(modelo_arvore)
        treino_c.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_treino_c, y_treino_c)
        teste_c.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_teste_c, y_teste_c)
        
        modelo_arvore = DecisionTreeClassifier(max_depth=i)
        modelo_arvore.fit(X_treino_gc, y_treino_gc)
        modelos_gc[i-profund[0]].append(modelo_arvore)
        treino_gc.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_treino_gc, y_treino_gc)
        teste_gc.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_teste_gc, y_teste_gc)

### Plot interativo da Árvore de Decisão

In [17]:
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt


# Plot interativo da árvore de decisão
def plot_arvore(alvo_mec_acao: str = 'acetylcholine_receptor', profundidade: int = 3, indicador: str = 'gene') -> None:
    """Gráfico de árvore dos experimentos.
      
    argumentos:
    alvo_mec_acao -- alvo do mecanismo de ação (padrão: 'adenylyl_cyclase')
    profundidade -- profundidade máxima na Árvore de Decisão (padrão: 3)
    indicador -- indicador para relação com os alvos do mecanismo de ação (padrão: 'gene')
    """
    col_idx = dados_resultados_tratados.columns.get_loc(alvo_mec_acao) - 1

    plt.style.use('seaborn-dark-palette')
    fig, ax = plt.subplots(figsize=(12,8))

    if indicador == 'gene':
        tree.plot_tree(
            modelos_g[profundidade-profund[0]][col_idx],
            ax=ax,
            fontsize=8,
            filled=True,
            rounded=True,
            feature_names=pd.DataFrame(X_treinos_g[col_idx][0]).columns,
            class_names=np.unique(y_treinos_g[col_idx]),
        )
        ax.set_title(f'Alvo do mecanismo de ação: {alvo_mec_acao} ; Acurácia no treinamento: {round(treino_g.loc[profundidade, alvo_mec_acao]*100, 2)}% ; Acurácia no teste: {round(teste_g.loc[profundidade, alvo_mec_acao]*100, 2)}%',fontdict={'fontsize':'larger','color':'black'})

    elif indicador == 'tipo celular':
        tree.plot_tree(
            modelos_c[profundidade-profund[0]][col_idx],
            ax=ax,
            fontsize=8,
            filled=True,
            rounded=True,
            feature_names=pd.DataFrame(X_treinos_c[col_idx][0]).columns,
            class_names=np.unique(y_treinos_c[col_idx]),
        )
        ax.set_title(f'Alvo do mecanismo de ação: {alvo_mec_acao} ; Acurácia no treinamento: {round(treino_c.loc[profundidade, alvo_mec_acao]*100, 2)}% ; Acurácia no teste: {round(teste_c.loc[profundidade, alvo_mec_acao]*100, 2)}%',fontdict={'fontsize':'larger','color':'black'})
        
    elif indicador == 'ambos':
        tree.plot_tree(
            modelos_gc[profundidade-profund[0]][col_idx],
            ax=ax,
            fontsize=8,
            filled=True,
            rounded=True,
            feature_names=pd.DataFrame(X_treinos_gc[col_idx][0]).columns,
            class_names=np.unique(y_treinos_gc[col_idx]),
        )
        ax.set_title(f'Alvo do mecanismo de ação: {alvo_mec_acao} ; Acurácia no treinamento: {round(treino_gc.loc[profundidade, alvo_mec_acao]*100, 2)}% ; Acurácia no teste: {round(teste_gc.loc[profundidade, alvo_mec_acao]*100, 2)}%',fontdict={'fontsize':'larger','color':'black'})
    fig.show()


# Menus interativos
plot_arvor_complet_interativo = interactive(
    plot_arvore,
    alvo_mec_acao=alvos_mec_acao_observados,
    profundidade=list(range(profund[0],profund[1])),
    indicador=['gene','tipo celular','ambos'],
)
output = plot_arvor_complet_interativo.children[-1]
plot_arvor_complet_interativo

interactive(children=(Dropdown(description='alvo_mec_acao', index=3, options=('5-alpha_reductase', '11-beta-hs…

### DataFrame interativo dos mecanismos de ação

In [19]:
# DataFrame interativo dos mecanismos de ação com crosstab
def plot_df(alvo_m_a: str = 'acetylcholine_receptor', tratamento: str = 'com_droga') -> None:
    """DataFrame interativo de mecanismos de ação relevantes sobre um alvo e tratamento determinados.
      
    argumentos:
    alvo_m_a -- alvo do mecanismo de ação (padrão: 'adenylyl_cyclase')
    tratamento -- tratamento aplicado (padrão: com_droga)
    """    
    print('')
    print(f'Tabala de casos de mecanismos de ação sobre o alvo: {alvo_m_a}; com tratamento: {tratamento}')
    print('Pode levar um instante para atualizar.')
    composto_to_drop = []
    for composto in dados['composto'].unique():
        if dados_resultados_tratados_completos[dados['composto'] == composto][alvo_m_a].unique() == 'NA':
            composto_to_drop.append(composto)

    compostos_relevantes = np.setdiff1d(dados['composto'].unique(), composto_to_drop)

    print('Tabela atualizada!')
    display(pd.crosstab(
        [
            dados_resultados_tratados_completos[alvo_m_a],
            dados[dados['tratamento'] == tratamento]['dose'],
            dados[dados['tratamento'] == tratamento]['tempo'],
        ],
        dados[dados['composto'].isin(compostos_relevantes)]['composto'],
    ))


# Menus interativos
plot_df_interativo = interactive(
    plot_df,
    alvo_m_a=dados_resultados_tratados_completos.loc[:,'5-alpha_reductase':].columns,
    tratamento=['com_droga','com_controle'],
)
output = plot_df_interativo.children[-1]
plot_df_interativo

interactive(children=(Dropdown(description='alvo_m_a', index=3, options=('5-alpha_reductase', '11-beta-hsd1', …